In [23]:
# imports
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from gan import Generator, Discriminator, GAN
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import pdb
import torch
from PIL import Image

# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# instantiate generator
generator = Generator()

# instantiate discriminator (pre-trained facial recognition model)
discriminator = Discriminator()

# construct GAN network in full
gan = GAN(generator, discriminator)

100.0%


In [29]:
# load unmasked faces from FFHQ
num_faces = 10
unmasked_faces = torch.zeros((num_faces, 3, 128, 128))
mtcnn = MTCNN(image_size=128)
for face_id in range(num_faces):
    face_id_text = str(100000 + face_id)[1:]
    image_path = '../data/unmasked/{}.png'.format(face_id_text)
    face_image = mtcnn(Image.open(image_path))
    unmasked_faces[face_id] = face_image
    
# compute and store unmasked discriminator embeddings
gan.compute_unmasked_embeddings(unmasked_faces)

In [ ]:
# load masked faces from MaskedNet
masked_faces = torch.zeros((num_faces, 3, 128, 128))
mtcnn = MTCNN(image_size=128)
for face_id in range(num_faces):
    face_id_text = str(100000 + face_id)[1:]
    image_path = '../data/masked/{}_Mask.jpg'.format(face_id_text)
    face_image = mtcnn(Image.open(image_path))
    masked_faces[face_id] = face_image

In [ ]:
# train CycleGAN for projecting mask designs onto faces
# load_path = None
projector = CycleGAN()
projector.train(num_epochs=10, faces=masked_faces)

In [ ]:
# train
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    # sort data into minibatches
    minibatches = batch_data()
    
    # train on each minibatch
    for batch in minibatch:
        # get data
        faces, ids = batch
    
        # generate mask design
        mask = gan.generator()
    
        # project design onto masked images
        faces = gan.project(mask, faces)
        
        # 